In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [ ]:
final = pd.DataFrame()

# 30일, 31일 월 분리
# 2월은 따로 생각해야 함
months_30 = [4, 6, 9, 11]
months_31 = [1, 3, 5, 7, 8, 10, 12]

for year in range(2008, 2021):   # 여기 바꾸기
    for month in range(1, 13): 
        if month in months_30: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3012&STNM={}'.format(year, month, where), verify=False)
        elif month in months_31: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3112&STNM={}'.format(year, month, where), verify=False)
        else:   # 2월인 경우
            if year % 4 == 0:   # 윤년인 경우 따로 분리 - 29일까지 있음
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2912&STNM={}'.format(year, month, where), verify=False)
            else:   # 28일까지 있는 경우
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2812&STNM={}'.format(year, month, where), verify=False)
        
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        title = soup.select('body > h2')  # title은 하나씩 있음
        data = soup.select('body > pre')  # pre는 하나 건너 하나씩 concat 해야함

        idx_data = 0
        for idx in range(len(title)): 
            data_df = pd.DataFrame(data[idx_data])
            data_df[0] = data_df[0].str.replace("-----------------------------------------------------------------------------", "")
            data_df_2 = data_df[0].str.split('\n', expand=True)

            data_df_2.drop([0, 1, 2, 3, 4], axis = 1, inplace = True)

            data_df_2 = data_df_2.transpose()
            title_tmp = str(title[idx])
            data_df_2['date'] = title_tmp[4:54]   # upton 기준 [4:51] # gray 기준 [4:50] # Sterling 기준 [4:54]
            
            final = pd.concat([final, data_df_2], axis = 0)

            idx_data += 2

In [9]:
response = requests.get('https://aqs.epa.gov/data/api/monitors/byState?email=zio0323gm@gmail.com&key=indigowren21&param=42401&bdate=20150501&edate=20150502&state=15',verify=False)

response.text

c:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aqs.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


'{\n  "Header": [\n    {\n      "status": "Success",\n      "request_time": "2022-09-30T01:30:44-04:00",\n      "url": "https://aqs.epa.gov/data/api/monitors/byState?email=zio0323gm@gmail.com&key=indigowren21&param=42401&bdate=20150501&edate=20150502&state=15",\n      "rows": 23\n    }\n  ],\n  "Data": [\n    {\n      "state_code": "15",\n      "county_code": "003",\n      "site_number": "1001",\n      "parameter_code": "42401",\n      "poc": 6,\n      "parameter_name": "Sulfur dioxide",\n      "open_date": "1992-10-16",\n      "close_date": null,\n      "concurred_exclusions": null,\n      "dominant_source": null,\n      "measurement_scale": "NEIGHBORHOOD",\n      "measurement_scale_def": "500 M TO 4KM",\n      "monitoring_objective": "POPULATION EXPOSURE",\n      "last_method_code": "060",\n      "last_method_description": "INSTRUMENTAL - PULSED FLUORESCENT",\n      "last_method_begin_date": "1992-10-16",\n      "naaqs_primary_monitor": null,\n      "qa_primary_monitor": null,\n     

In [11]:
requests.get('https://aqs.epa.gov/data/api/sampleData/byState?email=zio0323gm@gmail.com&key=indigowren21&param=45201&bdate=19950515&edate=19950515&state=42').text

'{\n  "Header": [\n    {\n      "status": "Success",\n      "request_time": "2022-09-30T01:31:20-04:00",\n      "url": "https://aqs.epa.gov/data/api/list/states?email=zio0323gm@gmail.com&key=indigowren21",\n      "rows": 56\n    }\n  ],\n  "Data": [\n    {\n      "code": "01",\n      "value_represented": "Alabama"\n    },\n    {\n      "code": "02",\n      "value_represented": "Alaska"\n    },\n    {\n      "code": "04",\n      "value_represented": "Arizona"\n    },\n    {\n      "code": "05",\n      "value_represented": "Arkansas"\n    },\n    {\n      "code": "06",\n      "value_represented": "California"\n    },\n    {\n      "code": "08",\n      "value_represented": "Colorado"\n    },\n    {\n      "code": "09",\n      "value_represented": "Connecticut"\n    },\n    {\n      "code": "10",\n      "value_represented": "Delaware"\n    },\n    {\n      "code": "11",\n      "value_represented": "District Of Columbia"\n    },\n    {\n      "code": "12",\n      "value_represented": "Flori